In [3]:
import time
import sys 
import os
sys.path.append(os.path.abspath('.')) 
from src.RemoteControl import RemoteControl

HOST = '192.168.4.245'  # The smartphone's IP address


def main():
    # example class usage
    # constructor starts the connection
    remote = RemoteControl(HOST)
    print("Connected")
    
    #print("Magnetometer data length: %d" % len(magnetic_data))
    #with open("magnetic.csv", "w+") as imu_file:
    #    imu_file.writelines(magnetic_data)
    
    phase, duration, exp_time = remote.start_video()
    print("%d %f" % (exp_time, duration))
    time.sleep(5)
    remote.stop_video()

    # receives last video (blocks until received)
    start = time.time()
    filename = remote.get_video(want_progress_bar=True)
    end = time.time()
    print("elapsed: %f" % (end - start))
    print('Closing connection')
    remote.close()


if __name__ == '__main__':
    main()


Connected


KeyboardInterrupt: 

In [ ]:
import time
import sys
import os
import threading
import tkinter as tk
from tkinter import messagebox, scrolledtext

sys.path.append(os.path.abspath('.'))
from src.RemoteControl import RemoteControl

HOST = '192.168.4.245'  # The smartphone's IP address

class RemoteControlGUI:
    def __init__(self, master):
        self.master = master
        master.title("Remote Video Control")

        self.remote = None
        self.recording = False

        self.status_area = scrolledtext.ScrolledText(master, height=10, width=50, state='disabled')
        self.status_area.pack(padx=10, pady=10)

        self.start_button = tk.Button(master, text="Start Recording", command=self.start_recording)
        self.start_button.pack(padx=10, pady=5)

        self.stop_button = tk.Button(master, text="Stop Recording", command=self.stop_recording, state=tk.DISABLED)
        self.stop_button.pack(padx=10, pady=5)

        self.quit_button = tk.Button(master, text="Quit", command=master.quit)
        self.quit_button.pack(pady=10)

    def log(self, message):
        self.status_area.config(state='normal')
        self.status_area.insert(tk.END, message + "\n")
        self.status_area.see(tk.END)
        self.status_area.config(state='disabled')
        print(message)

    def start_recording(self):
        def thread_func():
            try:
                self.remote = RemoteControl(HOST)
                self.log("Connected to device.")

                phase, duration, exp_time = self.remote.start_video()
                self.log(f"Started video: exposure {exp_time}, duration {duration:.2f}")
                self.recording = True

                self.start_button.config(state=tk.DISABLED)
                self.stop_button.config(state=tk.NORMAL)

            except Exception as e:
                self.log(f"Error starting recording: {e}")
                messagebox.showerror("Error", str(e))

        threading.Thread(target=thread_func).start()

    def stop_recording(self):
        def thread_func():
            try:
                if not self.remote:
                    self.log("Not connected.")
                    return

                self.remote.stop_video()
                self.log("Stopped video. Downloading...")

                start = time.time()
                filename = self.remote.get_video(want_progress_bar=True)
                end = time.time()
                self.log(f"Video saved to {filename}. Elapsed: {end - start:.2f} seconds")

                self.remote.close()
                self.log("Connection closed.")

                self.start_button.config(state=tk.NORMAL)
                self.stop_button.config(state=tk.DISABLED)
                self.recording = False

            except Exception as e:
                self.log(f"Error stopping recording: {e}")
                messagebox.showerror("Error", str(e))

        threading.Thread(target=thread_func).start()


# if __name__ == "__main__":
root = tk.Tk()
app = RemoteControlGUI(root)
root.mainloop()
